In [88]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [89]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
df = pd.read_csv('streamer_df')

# trying to scrape historical data

## Scrape data # ez

In [130]:
def time_assignment(x):
    '''Support function to classify streams by time when they are on air'''
    midnight = 24
    morning = 6 
    noon = 12 
    evening = 18
    if morning <= x.hour < noon:
        return 1
    elif noon <= x.hour < evening:
        return 2
    elif evening <= x.hour < midnight:
        return 3
    else:
        return 0

In [129]:
def parse_streams(usernames):
    '''For all usernames input as a list, retrieve their stats for all streams and classify by daytime'''
    import time
    t=1
    count = 0
    df_all = pd.DataFrame(columns=['Stream', 'Duration', 'AVG Viewers', 'MAX Viewers', 'Followers Gain', 'Live Views', 'Title', 'Games', 'User'])
    for user in usernames:
        count += 1
        url = f"https://twitchtracker.com/{user}/streams"
        headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        df = pd.read_html(str(soup))[0]
        df['User'] = f'{user}'
        temp_df = pd.concat([df_all, df])

        temp_df['Stream'] = pd.to_datetime(temp_df['Stream'])
        temp_df['Duration'] = temp_df['Duration'].apply(lambda x: int(x))
        temp_df['AVG Viewers'] = temp_df['AVG Viewers'].apply(lambda x: int(x))
        temp_df['MAX Viewers'] = temp_df['MAX Viewers'].apply(lambda x: int(x))
        temp_df['Followers Gain'] = temp_df['Followers Gain'].apply(lambda x: int(x))
        temp_df['Live Views'] = temp_df['Live Views'].apply(lambda x: int(x))
        new_dates, new_times = zip(*[(d.date(), d.time()) for d in temp_df['Stream']])
        df_clean = temp_df.assign(new_date=new_dates, new_time=new_times)
        df_clean = df_clean[['User', 'new_date', 'new_time', 'Duration', 'AVG Viewers', 'MAX Viewers', 'Followers Gain', 'Live Views', 'Title']]
        df_clean['daytime_classifier'] = df_clean['new_time'].apply(lambda x: time_assignment(x))
        print(count)
        time.sleep(t)
    return df_clean
# df.to_csv(r'/Users/home/code/Awesimous/StarFish/notebooks/top100_streams', index=False)

In [127]:
temp_df = parse_streams(['agurin'])

1


## Trying to match somehow the time of stream with the game on the twitchtracker page

In [19]:
# filter for only agurin user
agurin = temp_df[temp_df['User'] == 'agurin']

zeiten = agurin['Stream'].to_list()

zeiten

In [70]:
## not working out ...

url = f"https://twitchtracker.com/agurin/streams"
headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

soup.findAll(class_='games hidden-xs hidden-sm')

[img.text for td in soup.find_all('img title', class_='games hidden-xs hidden-sm')]

soup.find_all('img title', class_='games hidden-xs hidden-sm')

soup.findAll('2017-08-14')

soup.find_all('a').find('src').get('title')

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?